The example script below demonstrates how a randomized plate layout 
can be generated from an initial description (see input_file.csv) of the experiment 

In [4]:
import pandas as pd
from datarail.experimental_design import process_assay as pa
from datarail.experimental_design import hpdd_utils as hu

In [3]:
# construct mapping between wells and treatment conditions
df_well = pa.construct_well_level_df('input_file.csv')
df_well = pa.add_dmso_control(df_well)

KeyError: 'exclude_doses'

In [3]:
# define drug and dose to be used as harsh treatment for plate barcode
fingerprint_drug = 'Staurosporine'
fingerprint_dose = 1

In [4]:
# create randomized plate layout for 3 replicates
plate_names = ['RL_A', 'RL_B', 'RL_C']

In [5]:
dfr = pa.randomize_wells(df_well, fingerprints=plate_names,
                         fingerprint_drug=fingerprint_drug,
                         fingerprint_dose=fingerprint_dose)

In [6]:
dfr.head()


,agent,concentration,identifier,plate,role,well
0,Staurosporine,1.0,Staurosporine_0,RL_A,fingerprint,A01
1,Staurosporine,1.0,Staurosporine_1,RL_A,fingerprint,A02
2,,0.0,0,RL_A,,A03
3,,0.0,0,RL_A,,A04
4,Staurosporine,1.0,Staurosporine_2,RL_A,fingerprint,A05


In [7]:
# load a file (see drug_stock.csv) with name of the agents and the corresponding stock concentrations
# The name of the agents in the file should match those in design dataframe

In [8]:
df_stock = pd.read_csv('drug_stock.csv')


In [9]:
# If the D300 is going to be used, run the code below to generate the hpdd file

In [10]:
hu.export_hpdd(dfr, df_stock, 'RL_D300.hpdd')